In [1]:
# create user object with all information
# define function that separates out user information and provides recommendations based on what is available
# perhaps calibrate n for CB depending on what CF returns
# categories...?

In [2]:
%run CB.ipynb

In [4]:
%run CF.ipynb

# Weighing Recommendations: #
1) If user has both kinds of information
2) If user has only one
3) If user has incomplete information in both sections???



In [ ]:
# Widget: https://www.youtube.com/watch?v=jKjFbvWugZA

# Recommender Interface #

In [ ]:
def access_recommender(user_info):
    pass 

# Collaborative Filtering Recommender Testing #

## Define Testing Functions ##


In [ ]:
# https://youtu.be/A0K3K4fOvY8 - precision and recall at k

In [172]:
def get_hit_ratio(testset, n):
    '''
        
    '''
    
    hits = 0
    n_records = 0
    top_n_rec_results = {}
    
    # Gather recommendations
    for user in testset.index:
#         print(f"USER: {user}")
        actual = testset.loc[user, testset.columns[-1]]
#         print(actual)
        rec_list = CF_recommender(user, n=n, testset=testset.iloc[:,:-1])
        
        hits_per_user = 0
        for course_taken in actual:
#             print(f"course: {course_taken}")
            if (course_taken in rec_list):
                hits_per_user +=1
            
            # Add records manually
            #  This approach allows for mixed testsets
            n_records += 1
#         print(f"hits per user: {hits_per_user}")

            
        hits += hits_per_user
        
        # Build object
        top_n_rec_results[user] = {"actual": actual,
                                   "recommended": rec_list,
                                   "hits": hits_per_user
                                }
    
    # Process results
    hit_ratio = hits/n_records

    return top_n_rec_results, hit_ratio
        
def tabulate_hit_ratio_n_range (rec_results, n_start, n_stop):
    
    for n in range(n_start, n_stop+1):
        print("-------------------------------------------------")
        print(f"Statistics for n={n}")
        
        batch_hit_count = 0
        course_count = 0
        for user in rec_results:   
#             print(user)
            batch_list = list(rec_results[user]["recommended"].keys())[:n]
            for course in rec_results[user]["actual"]:
                # Record count for course completed
                course_count +=1 
                
                if course in batch_list:                    
#                     print(f"{course} in {batch_list}")
                    batch_hit_count +=1 
                
        print(f"Number of hits: {batch_hit_count}")
#         print(f"Number of courses: {course_count}")
        print(f"Hit ratio: {round(batch_hit_count/course_count, 4)}")
    
    print("-------------------------------------------------")
            
def get_MRR_at_n (rec_results, n=10):
    
    # Define list to hold placement of each
    #  relevant recommendation
    RR = []
    
    # Loop through recommendations
    for user in list(rec_results.keys()):
#         print(user)
        # Loop through each of the completed courses
        actual = rec_results[user]["actual"]
        for course in actual:
            found = False # define a found flag
            # Search first n recommended items to find relevant item
            for i in range(n):
                # When relevant item is found, add its reciprocal rank to the list
                if (course == list(rec_results[user]["recommended"].keys())[i]):
                    # Record reciprocal of ranked recommendation
                    found = True
                    RR.append(1/(i+1))
            
            if not found:
                RR.append(0)
                    
                    
    MRR = sum(RR)/len(RR)
    
    return MRR

## Prepare Test Sets ##

### *Test Set A* ###
Filter database for all users who have completed  one course and check predictions by hiding the completed course information and measure the performance of the CF recommender in relation to the actual course completed by each of these users.

In [5]:
testsetA = test_pt[test_pt["sum"]==1].copy()
testsetA

course_id,14.73x,2.01x,3.091x,6.002x,6.00x,7.00x,8.02x,8.MReV,CB22x,CS50x,ER22x,PH207x,PH278x,sum
userid_DI,,,,,,,,,,,,,,
H130000071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
H130000715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
H130000944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
H130001577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
H130002019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M130595975,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
M130596958,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
M130597250,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [6]:
user_list_testsetA = list(testsetA.index)
# user_list_testsetA

In [7]:
# Store values before 
testsetA["course_completed_by_user"] = [list(testsetA.columns[np.where(testsetA.loc[user, testsetA.columns[:-1]]==1)[0]]) for user in testsetA.index]

In [8]:
# Change all 1 values to 0
for col in testsetA.columns[:-1]:
    testsetA[col] = np.where(testsetA[col]==1, 0, testsetA[col])

In [9]:
testsetA

course_id,14.73x,2.01x,3.091x,6.002x,6.00x,7.00x,8.02x,8.MReV,CB22x,CS50x,ER22x,PH207x,PH278x,sum,course_completed_by_user
userid_DI,,,,,,,,,,,,,,,
H130000071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,[CB22x]
H130000715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,[CS50x]
H130000944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,[ER22x]
H130001577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,[CB22x]
H130002019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,[PH207x]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M130595975,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,[6.002x]
M130596958,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,[6.00x]
M130597250,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,[6.00x]


In [10]:
# testsetA

### *Test Set B* ###
Filter database of all users who have completed two courses (i.e., the maximum number of courses completed by any user in the test) and check predictions in two ways:
1) hide all courses and check recommendations for two course
2) hide one course at a time and check recommendations

In [116]:
testsetB = test_pt[test_pt["sum"]==2].copy()

In [117]:
testsetB_user_list = testsetB.index
# testsetB_user_list

In [118]:
testsetB

course_id,14.73x,2.01x,3.091x,6.002x,6.00x,7.00x,8.02x,8.MReV,CB22x,CS50x,ER22x,PH207x,PH278x,sum
userid_DI,,,,,,,,,,,,,,
H130001147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0
H130011763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0
H130019058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,2.0
H130030879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0
H130032038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M130550356,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
M130557833,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
M130561878,1.0,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [119]:
# Store values before rewriting table
courses_selected = [list(testsetB.columns[np.where(testsetB.loc[user, testsetB.columns[:-1]]==1)[0]]) for user in testsetB_user_list]

# Transform course information into np.array for ease of processing
# courses_selected = np.array(courses_selected) 

# Check output
# courses_selected

In [120]:
# Add course information to dataframe before modifying values
testsetB["actual_course"] = courses_selected
# testsetB["actual_course_2"] = courses_selected[:, 1]

In [122]:
# Change all non-zero values to 0
for col in testsetB.columns[:-1]:
    testsetB[col] = np.where((testsetB[col]==1) | (testsetB[col]==2), 0, testsetB[col])

In [123]:
testsetB

course_id,14.73x,2.01x,3.091x,6.002x,6.00x,7.00x,8.02x,8.MReV,CB22x,CS50x,ER22x,PH207x,PH278x,sum,actual_course
userid_DI,,,,,,,,,,,,,,,
H130001147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,"[ER22x, PH278x]"
H130011763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,"[ER22x, PH278x]"
H130019058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,"[PH207x, PH278x]"
H130030879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,"[ER22x, PH278x]"
H130032038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,"[PH207x, PH278x]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M130550356,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[6.002x, 8.02x]"
M130557833,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[6.002x, 8.02x]"
M130561878,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"[14.73x, 6.00x]"


### Other measures ###
* MAP
* MRR
* NDCG

In [153]:
# https://www.evidentlyai.com/ranking-metrics/evaluating-recommender-systems
# https://www.evidentlyai.com/ranking-metrics/mean-reciprocal-rank-mrr

### *Calculate Hit Ratio for Testset A and Tabulate Results* ###

In [49]:
testsetA_full_recs, testsetA_overall_hit_ratio = get_hit_ratio(testsetA, n=10)

In [175]:
get_MRR_at_n(testsetA_full_recs, n=10)

0.5165358133016474

In [180]:
get_MRR_at_n(testsetA_full_recs, n=5)

0.4927191298784399

In [179]:
get_MRR_at_n(testsetB_full_recs, n=10)

0.36785714285714294

In [181]:
get_MRR_at_n(testsetB_full_recs, n=5)

0.32458333333333333

In [127]:
tabulate_hit_ratio_n_range(testsetA_full_recs, 1,10)

-------------------------------------------------
Statistics for n=1
Number of hits: 845
Hit ratio: 0.3244
-------------------------------------------------
Statistics for n=2
Number of hits: 1323
Hit ratio: 0.5079
-------------------------------------------------
Statistics for n=3
Number of hits: 1627
Hit ratio: 0.6246
-------------------------------------------------
Statistics for n=4
Number of hits: 1863
Hit ratio: 0.7152
-------------------------------------------------
Statistics for n=5
Number of hits: 2059
Hit ratio: 0.7904
-------------------------------------------------
Statistics for n=6
Number of hits: 2209
Hit ratio: 0.848
-------------------------------------------------
Statistics for n=7
Number of hits: 2317
Hit ratio: 0.8894
-------------------------------------------------
Statistics for n=8
Number of hits: 2398
Hit ratio: 0.9205
-------------------------------------------------
Statistics for n=9
Number of hits: 2460
Hit ratio: 0.9443
------------------------------

### *Calculate Hit Ratio for Testset B* ###

In [62]:
testsetB_full_recs, testsetB_overall_hit_ratio = get_hit_ratio(testsetB, n=10)

In [128]:
tabulate_hit_ratio_n_range(testsetB_full_recs, 2,10)

-------------------------------------------------
Statistics for n=2
Number of hits: 57
Hit ratio: 0.285
-------------------------------------------------
Statistics for n=3
Number of hits: 89
Hit ratio: 0.445
-------------------------------------------------
Statistics for n=4
Number of hits: 110
Hit ratio: 0.55
-------------------------------------------------
Statistics for n=5
Number of hits: 125
Hit ratio: 0.625
-------------------------------------------------
Statistics for n=6
Number of hits: 138
Hit ratio: 0.69
-------------------------------------------------
Statistics for n=7
Number of hits: 156
Hit ratio: 0.78
-------------------------------------------------
Statistics for n=8
Number of hits: 166
Hit ratio: 0.83
-------------------------------------------------
Statistics for n=9
Number of hits: 181
Hit ratio: 0.905
-------------------------------------------------
Statistics for n=10
Number of hits: 191
Hit ratio: 0.955
-------------------------------------------------


# Content-Based Recommender Testing #

<u>Preceeding Steps (CB.ipynb)</u>:
* CB dataframe has been split into CB_train_df and CB_test_df
* TF-IDF vectorizer has been initiated
* vectorizer was fit to the keywords in CB_train_df and TF-IDF matrix was created
* extracted keywords in the test set

### *Define Testing Function* ###

In [203]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

def CB_testing (encoded_course_matrix=tfidf, test_data=test_keywords, vectorizer=vectorizer):
    '''
        Define function for testing the CB recommender given a set of test keywords. The function first 
        transforms test keywords into TF-iDF vectors and then calculates the cosine similarity between
        each test vector and each training vector in the test set, storing the results in the shape of 
        a matrix. The testset defined above is based on CB_test_df in CB.ipynb

        Inputs:
            - tfidf_matrix : matrix created by fitting and transforming the "training set"
                               (supplied input is the matrix based on the transformed CB_train_df in CB.ipynb)
            - test_data    : a (nested) array containing keywords from each of the testsets
                               (supplied input is test_keywords, which have been extracted form CB_test_df)
            - vectorizer   : vectorizer with which to transform test data
                               (supplied fectorizer is the TfidfVectorizer from CB.ipynb)
        
        Outputs:
            - results      : matrix containing a measure of the cosine similarity between each test vector
                             and each training vector
    '''
    
    
    # Define dimensions of the results matrix based on the size of the 
    #  training and test sets
    m = encoded_course_matrix.shape[0]
    n = len(test_keywords)
#     print(m,n)

    # Transform test data into vectors for further processing
    test_vecs = [vectorizer.transform([record[1]]) for record in test_keywords] # encode the second element of the tuple which contains the keywords themselves
    #     print(test_vecs)
            
    # Define vaiable to hold test rezults by specifying its dimensions
    results = np.empty((m,n))   

    
#     print(cosine_similarity(tfidf_matrix[0], test_vecs[0]))
    
    # Calculate the cosine similarity between each training and test vector
    for i, train_vec in enumerate(encoded_course_matrix):
        for j, test_vec in enumerate(test_vecs):
            results[i][j] = cosine_similarity(train_vec, test_vec)[0][0]
            
            
    return results

# Alternate testing function
# def test_cosine_similarity (test_set=test_keywords, encoded_course_matrix=tfidf):
   
#     # Create matrix to store results
#     results = np.empty((tfidf.shape[0], len(test_set)))
    
#     # Encode the test set
#     test_vecs = [encode_query(keywords[1]) for keywords in  test_set] # encode the second element of the tuple which contains the keywords themselves
    
#     # Populate results matrix in accordance with pre-set structure
#     for i, course in enumerate(encoded_course_matrix):
#         for j,testvec in enumerate(test_vecs):
#             results[i][j] = cosine_similarity(course,testvec)[0][0]
    
#     return results

def CB_tabulate_results(results, test_set = test_keywords, training_set = training_keywords, encoded_course_matrix=tfidf, n=10):
    '''
        Display information about each item in the test set and the recommendations issued by the subsystem
        in a quasi-tabular manner for ease of inspection
        
        
    '''
    # Loop through each vector in the test_set
    for i in range(results.shape[1]):
        
        # Isolate the measured closeness to all vectors in the course matrix
        top_10_recs = np.argsort(results[:,i])[:-n-1:-1]
        
        # Output text
        print("-------------------------------------------------------------------------")
        print(f"Test_keywords {i}: ")
        print(test_set[i], CB_test_df.loc[test_set[i][0], "Course Number"],CB_test_df.loc[test_set[i][0], "Institution"])
        
        print("\nRecommendations:")
        for rec in top_10_recs:
#             print(rec)
            
            print(training_set[rec], CB_train_df.iloc[rec, 1]) #, train_df.loc[rec, "Course Number"], train_df.loc[rec, "Institution"])
                                                                                           

def CB_subject_accuracy_at_k(results, test_set=test_keywords, training_set=training_keywords, n=5):
    '''
        Calculate the accuracy with which the recommender chooses courses that have the same subject for any value of n.
        
        Inputs:
        
        Outputs: 
        
    '''
    
    # Define variable to store accuracy per test sample
    accuracy_per_sample = {}
    accuracy_per_sample[n] = {} 
    
    # Number of samples
    n_samples = results.shape[1]
    
    
    # Loop through all the test cases
    for idx in range(n_samples):
        top_n_recs = np.argsort(results[:,idx])[:-n-1:-1]
        
        # Isolate training subject
        target_subject = test_set[idx][1][-1]
#         print(test_set[idx][1])
#         print(f"idx: {idx}, target subject: {target_subject}")
            
        # Define count variable
        count = 0
        for rec in top_n_recs:
#             print(training_set[rec])
#             print(CB_train_df.iloc[rec])
#             print(f"pred: {training_set[rec][1][-1]}")
            if (training_set[rec][1][-1]==target_subject):
                count +=1

        accuracy_per_sample[n][idx] = count/n
        
    return accuracy_per_sample

def CB_precision_at_k(results, k=5):
    # of the number of items recommended, how many are relevant
    '''
        This function adapts the precision @ k measure in order to provide a measure of the relevance of each recommendation
        by calculating the precision of individual keywords and then averaging it out across all recommendations. In short, 
        this function measures the relevance of each recommendation by computing the number of relevant keywords it contains
        in relation to the query and then averaging out this measure across the entire query set.
    '''
    word_stats_p_at_k = {}
    avg_p_at_k = {}

    # Loop through results
    for i in range(results.shape[1]):

            # Isolate the measured closeness to all vectors in the course matrix
            top_k_recs = np.argsort(results[:,i])[:-k-1:-1]
#             print(top_k_recs)
            
            # Extract keywords in query
            test_words = test_keywords[i][1]
#             print(f"Test subject: {test_words}, {len(test_words)}")

            # Extract keywords in recommendation
            rec_words = [training_keywords[rec][1] for rec in top_k_recs]
    #         print(f"Rec subject: {rec_words}")
            
            # Create variable to hold results
            word_stats_p_at_k[i] = []
            # https://stackoverflow.com/questions/1388818/how-can-i-compare-two-lists-in-python-and-return-matches
            for rec in rec_words:
#                 print(f"Set: {set(test_words) & set(rec)}")
#                 print(rec)
                # Count the number of words that are relevant of the entire number of words
                #  returned in each recommendation
                word_stats_p_at_k[i].append(len(set(test_words) & set(rec))/len(rec))
            
            # Average out this measure across k recommendations
            avg_p_at_k[i] = sum(word_stats_p_at_k[i])/k
   
    return avg_p_at_k



def CB_recall_at_k(results, k=5):
    # of the total number of relevant items, how many of them were picked out in top k#
    #!!!!!!!!!!!!!!!!
    
    # Define variables to hold data
    word_stats_r_at_k = {}
    avg_r_at_k = {}

    for i in range(results.shape[1]):

        # Isolate the measured closeness to all vectors in the course matrix
        top_k_recs = np.argsort(results[:,i])[:-6:-1]
    #         print(top_k_recs)

        test_words = test_keywords[i][1]
        print(f"Test subject: {test_words}, {len(test_words)}")

        rec_words = [training_keywords[rec][1] for rec in top_k_recs]
    #         print(f"Rec subject: {rec_words}")

        word_stats_r_at_k[i] = []
        # https://stackoverflow.com/questions/1388818/how-can-i-compare-two-lists-in-python-and-return-matches
        for rec in rec_words:
            print(len(set(test_words) & set(rec)))
            word_stats_r_at_k[i].append(len(set(test_words) & set(rec))/len(test_words))

        avg_r_at_k[i] = sum(word_stats_r_at_k[i])/k
    #             print(rec)
    
    return avg_r_at_k

### *Process Test Set* ###

In [184]:
# Process similarity of test keywords to known keywords
results = CB_testing()

### *Calculate Percetange of Correct Subject Attirbution* ###
Because the courses in the test and training sets are different, the keywords for one course will never match another in their entirety unless a course has been acidentally miscategorized using a different course code or unless its course code has been modified over time. (These are very few instances in this paritcular dataset where it appears that the course providers decided to make a slight modification the course code of a subsequent offering.) However, it seems to me to be nevertheless useful to check the extent to which the keyword search yields courses in the same category because it gives us a measure not only of the extent to which the algoirthm works to narrow down recommendations but also the way in which this algorithm includes elements of serendipity.

In [197]:
for i in range(10):
    n=i+1
    accuracy = CB_subject_accuracy_at_k(results, n=n)

#     print(accuracy)
    print(f"Average accuracy at {n}: {sum(accuracy[n].values())/len(accuracy[n])}")

Average accuracy at 1: 1.0
Average accuracy at 2: 0.9736842105263158
Average accuracy at 3: 0.9298245614035089
Average accuracy at 4: 0.9210526315789473
Average accuracy at 5: 0.905263157894737
Average accuracy at 6: 0.8947368421052632
Average accuracy at 7: 0.8947368421052634
Average accuracy at 8: 0.8881578947368421
Average accuracy at 9: 0.8771929824561406
Average accuracy at 10: 0.8842105263157896


The decrease in this accuracy measure tells us, in effect, that the correct categories emerge within the first few recommendations and that opening up the recommender to more options can make it perform less optimally. Another way to phrase this would be to say that as the number of recommendations we ask it to generate increases, the system introduces more serendipity.

### *Precision at K* ###
Because a CB subsystem focused on finding proximity of keywords returns items that are relevant on a scale, I have decided to use precision as a way to measure the relevance of what is returned, but to do so by calculating the precision of the words returned by every recommendation and to average that out over *k* recommendations.

In [209]:
for k in range(10):    
    print("-------------------------------------------------------------------------")
    print(f"Precision at {k+1} for each sample:")
    p = CB_precision_at_k(results, k+1)
    
    print(p,'\n')
    avg_p = sum(p.values())/len(p.values())
    print(f"Average precision: {avg_p}")
    

-------------------------------------------------------------------------
Precision at 1 for each sample:
{0: 0.8571428571428571, 1: 0.8, 2: 1.0, 3: 0.5, 4: 1.0, 5: 0.75, 6: 0.75, 7: 0.7777777777777778, 8: 1.0, 9: 0.4, 10: 1.0, 11: 1.0, 12: 0.75, 13: 0.3333333333333333, 14: 0.6666666666666666, 15: 0.5454545454545454, 16: 1.0, 17: 0.5, 18: 1.0} 

Average precision: 0.7700197463355358
-------------------------------------------------------------------------
Precision at 2 for each sample:
{0: 0.8571428571428571, 1: 0.6857142857142857, 2: 0.8333333333333333, 3: 0.45, 4: 0.8333333333333333, 5: 0.5892857142857143, 6: 0.7083333333333333, 7: 0.7638888888888888, 8: 0.6666666666666666, 9: 0.4, 10: 0.7222222222222222, 11: 1.0, 12: 0.625, 13: 0.3333333333333333, 14: 0.6666666666666666, 15: 0.5852272727272727, 16: 1.0, 17: 0.35, 18: 1.0} 

Average precision: 0.6879025214551531
-------------------------------------------------------------------------
Precision at 3 for each sample:
{0: 0.8571428571

In [208]:
CB_recall_at_k(results)

Test subject: ['their', 'through', 'world', 'scriptures', 'hinduism', 'religions', 'hhdre'], 7
6
6
6
6
1
Test subject: ['nature', 'cellular', 'applications', 'solids', 'stem'], 5
4
4
2
1
1
Test subject: ['stress', 'behavior', 'mechanical', 'columns', 'transformations', 'materials', 'solids', 'beams', 'cellular', 'stem'], 10
4
4
3
4
3
Test subject: ['empire', 'last', 'hhdre'], 3
2
2
1
1
1
Test subject: ['society', 'business', 'self', 'transforming', 'ghss'], 5
5
2
2
1
1
Test subject: ['fantastique', 'first', 'nights', 'symphonie', 'hhdre'], 5
3
3
1
1
1
Test subject: ['getting', 'science', 'results', 'health', 'practical', 'care', 'roadmap', 'improvement', 'ghss'], 9
3
2
2
2
2
Test subject: ['ancient', 'epic', 'greek', 'signs', '6-11', 'iconography', 'hero', 'hours', '24', 'hhdre'], 10
7
6
4
4
6
Test subject: ['becoming', 'entrepreneur', 'ghss'], 3
3
1
1
1
1
Test subject: ['game', 'design', 'introduction', 'hhdre'], 4
2
2
1
1
1
Test subject: ['your', 'who', 'entrepreneurship', 'is', '101

{0: 0.7142857142857142,
 1: 0.4800000000000001,
 2: 0.36,
 3: 0.4666666666666666,
 4: 0.43999999999999995,
 5: 0.36,
 6: 0.24444444444444446,
 7: 0.5399999999999999,
 8: 0.4666666666666666,
 9: 0.35,
 10: 0.45714285714285713,
 11: 0.8,
 12: 0.37142857142857144,
 13: 0.3333333333333333,
 14: 0.2333333333333333,
 15: 0.3818181818181819,
 16: 0.6399999999999999,
 17: 0.4,
 18: 0.5}

{1: {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 16: 1.0, 17: 1.0, 18: 1.0}}
Average accuracy at 1: 1.0
{2: {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 16: 1.0, 17: 0.5, 18: 1.0}}
Average accuracy at 2: 0.9736842105263158
{3: {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 0.6666666666666666, 7: 1.0, 8: 1.0, 9: 0.6666666666666666, 10: 1.0, 11: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 16: 1.0, 17: 0.3333333333333333, 18: 1.0}}
Average accuracy at 3: 0.9298245614035089
{4: {0: 1.0, 1: 0.75, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 0.75, 7: 1.0, 8: 1.0, 9: 0.5, 10: 1.0, 11: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 16: 1.0, 17: 0.5, 18: 1.0}}
Average accuracy at 4: 0.9210526315789473
{5: {0: 1.0, 1: 0.8, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 0.8, 7: 1.0, 8: 1.0, 9: 0.4, 10: 0.8, 11: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15

In [188]:
CB_tabulate_results(results, n=5)

-------------------------------------------------------------------------
Test_keywords 0: 
(183, ['their', 'through', 'world', 'scriptures', 'hinduism', 'religions', 'hhdre']) HDS3221.5x HarvardX

Recommendations:
(166, ['their', 'christianity', 'through', 'world', 'scriptures', 'religions', 'hhdre']) HDS3221.2x
(178, ['their', 'through', 'islam', 'world', 'scriptures', 'religions', 'hhdre']) HDS3221.4x
(173, ['their', 'through', 'buddhism', 'world', 'scriptures', 'religions', 'hhdre']) HDS3221.3x
(162, ['their', 'through', 'traditions', 'world', 'scriptures', 'religious', 'religions', 'literacy', 'hhdre']) HDS3221.1x
(20, ['justice', 'hhdre']) ER22x
-------------------------------------------------------------------------
Test_keywords 1: 
(174, ['nature', 'cellular', 'applications', 'solids', 'stem']) 3.054.3x MITx

Recommendations:
(168, ['medicine', 'cellular', 'applications', 'solids', 'stem']) 3.054.2x
(159, ['properties', 'applications', 'engineering', 'solids', 'structures', '